In [1]:
import redis

In [2]:
db = redis.Redis(host='localhost', port=6379, db=14)

In [3]:
keys = db.keys()

In [4]:
len(keys)

51655

In [5]:
import json

In [6]:
keys[0]

b'ekotip'

In [7]:
db['agung']

b'{"definisi": ["[a.gung]  |  \\u0627\\u0762\\u0648\\u06a0Definisi : 1. (paling) besar, terutama: balai ~; layar ~; tiang ~; 2. yg tertinggi: pemerintah ~; 3. teristimewa, terkemuka, terutama: Allahyarham Tun Abdul Razak seorang pemimpin yg ~; Allahyarham P. Ramlee seniman ~ Malaysia; tamu ~; 4. yg terbuka kpd dan utk dihadiri oleh semua ahli (bkn mesyuarat, persidangan, dsb): persidangan ~ UMNO; mesyuarat ~ Kesatuan Sekerja DBP; mengagungkan menjadikan (menganggap dsb) agung, membesarkan, memuliakan, me\\u00adninggikan: al-Quran ~ ilmu pengetahuan dan memuliakan ahli ilmu; mengagung-agungkan selalu (sering kali) mengagungkan, sangat mengagungkan; keagungan kebesaran, kemuliaan, keting\\u00adgian: ~ Abdullah sbg pengarang terletak pd konsepsinya yg moden; keagungan kebesaran, kemuliaan, keting\\u00adgian: ~ Abdullah sbg pengarang terletak pd konsepsinya yg moden; pengagungan perbuatan dsb mengagungkan: antara yg menghalang perkembangan pesat bahasa Melayu ialah kekurangan bahan rujuk\\

In [8]:
d = json.loads(db[keys[0]])

In [9]:
from tqdm import tqdm

selected = {}

for k in tqdm(keys):
    d = json.loads(db[k])
    if not len(d['definisi']):
        continue
        
    selected[k.decode()] = d

100%|██████████████████████████████████████████████████████████████████████████████████| 51655/51655 [00:01<00:00, 29477.56it/s]


In [10]:
len(json.loads(db['agung'])['definisi'])

3

In [11]:
len(selected)

25862

In [12]:
import re
from unidecode import unidecode

alphabet = 'qwertyuiopasdfghjklzxcvbnm'
consonants = 'bcdfghjklmnpqrstvwxyz'
vowels = 'aeiou'
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

In [13]:
import enchant
d = enchant.Dict('en_US')

In [14]:
results = set()

for data in tqdm(selected.values()):
    
    texts = []
    for definisi in data['definisi']:
        texts.extend(definisi.split('Definisi : ')[1].split())

    if data['tesaurus']:
        texts.extend(data['tesaurus'])
    
    for k in range(len(texts)):
        
        t = texts[k]
        t = unidecode(t)
        t = t.replace('--', '-')
        for p in PUNCTUATION:
            t = t.replace(p, '')

        if all([c in consonants for c in t]):
            continue

        if len(t) <= 3:
            continue

        if len(re.findall(r'\d+', t)):
            continue

        if t[0] in '-':
            continue

        if t in ['yg', 'utk']:
            continue
            
        if t[-1] in '-':
            continue
        
        if 'dll' in t:
            continue
            
        t = t.lower()
        
        if d.check(t):
            continue
            
        if t in results:
            continue
            
        results.add(t)

100%|███████████████████████████████████████████████████████████████████████████████████| 25862/25862 [00:09<00:00, 2724.83it/s]


In [15]:
len(results)

47748

In [16]:
results

{'esah',
 'berfikirlah',
 'mengetahui',
 'selera',
 'salvator',
 'ber-bagai-bagailah',
 'asean',
 'dibabat',
 'mengerawang',
 'teater',
 'perintangan',
 'rcambah',
 'cirian',
 'seelok-elok',
 'mengubui',
 'penyekutuan',
 'pengitlakan',
 'soyak',
 'menjelejeh',
 'perenang',
 'pakistan',
 'dijilat',
 'berjalan',
 'menantang',
 'gendut',
 'masak',
 'mengoyakkan',
 'kekar',
 'tiup',
 'bertenggang',
 'kelesa',
 'trajektori',
 'lihat-melihat',
 'dikesumba',
 'menyumbat',
 'manau',
 'sempadanpengganas',
 'teraniayadan',
 'menusuk-nusuk',
 'sajarah',
 'sawi',
 'penyerobot',
 'yalah',
 'penyertaan',
 'honggek',
 'pedoman',
 'teksnya',
 'daldaru',
 'per-temuan',
 'gerangannya',
 'menggedangkan',
 'terimbau',
 'membancuh',
 'merembet-rembet',
 'merkusii',
 'amalan',
 'mematil',
 'kandasamy',
 'menyendiri',
 'beringan-ringan',
 'merupakan',
 'bercerancang',
 'curaian',
 'menganjur',
 'mucronata',
 'membangun',
 'berpulasan',
 'turunlah',
 'pereka-pereka',
 'dolichos',
 'dikurangkan',
 'keper-cayaa

In [17]:
with open('dbp-v6.json', 'w') as fopen:
    json.dump(selected, fopen)

In [20]:
with open('dbp-keys.json') as fopen:
    dbp_keys = json.load(fopen)

In [24]:
with open('malay-words-v6.json', 'w') as fopen:
    json.dump(list(set(results) - set(dbp_keys)), fopen)